In [ ]:
# from ortools.sat.python import cp_model
from ortools.linear_solver.pywraplp import Solver

Make a variable for each nurse patient pair
Keep track of the total number of patients each nurse sees
Maximize the acuity of each nurse for each patient type
Max and min number of patients per nurse
Need to create a dictionary for patients so that each type maps to a number of patients

In [ ]:
from typing import List

class NursePatientMatcher:
    
    def __init__(self, num_patients, num_nurses, max_patients, min_patients, patient_types, acuities):
        self.p: int = num_patients
        self.n: int = num_nurses
        self.max_patients_per_nurse: int = max_patients
        self.min_patients_per_nurse: int = min_patients
        self.patients_of_each_type: List[int] = patient_types
        self.patient_nurse_acuities: List[List[int]] = acuities

In [ ]:
def create_variables(self):
    model: Solver = self.model
    n: int = self.n
    p: int = self.p
    patient_types = self.patients_of_each_type
    max_patients: int = self.max_patients_per_nurse
    self.x = {}
    self.patients_per_nurse = {}
    self.patient_types = {}
    for nurse in range(n):
        for patient in range(p):
            self.x[nurse, patient] = model.IntVar(0, 1, f'x[Nurse: {nurse}, Patient: {patient}]')
    
    for nurse in range(n):
        self.patients_per_nurse[nurse] = model.IntVar(0, max_patients, f'Patients for Nurse {nurse}')
    
    prev = 0
    for t in range(patient_types):
        num_patients = patient_types[t]
        self.patient_types[t] = set(range(prev, prev + num_patients))
        prev = prev + num_patients

# Register this method with the solver class
NursePatientMatcher.create_variables = create_variables

In [ ]:
def bound_patients_per_nurse(self):
    x = self.x
    model: Solver = self.model
    n: int = self.n
    p: int = self.p
    max_patients: int = self.max_patients_per_nurse
    min_patients: int = self.min_patients_per_nurse
    for nurse in range(n):
        model.Add(sum(x[nurse,patient] for patient in range(p)) <= max_patients)
        model.Add(sum(x[nurse,patient] for patient in range(p)) >= min_patients)

# Register this method with the solver class
NursePatientMatcher.bound_patients_per_nurse = bound_patients_per_nurse

In [ ]:
def one_nurse_per_patient(self):
    x = self.x
    model: Solver = self.model
    n: int = self.n
    p: int = self.p
    for patient in range(p):
        model.Add(sum(x[nurse,patient] for nurse in range(n)) == 1)


# Register this method with the solver class
NursePatientMatcher.one_nurse_per_patient = one_nurse_per_patient

In [ ]:
def track_patients_per_nurse(self):
    x = self.x
    model: Solver = self.model
    n: int = self.n
    p: int = self.p
    patients_per_nurse = self.patients_per_nurse
    for nurse in range(n):
        model.Add(sum(x[nurse,patient] for patient in range(p)) == patients_per_nurse[nurse])


# Register this method with the solver class
NursePatientMatcher.track_patients_per_nurse = track_patients_per_nurse